In [1]:
import os
import getpass

if not os.environ.get("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key: ")

INDEX_NAME = "aion2-guide-rag"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"


In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

vector_store = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("Pinecone 벡터 저장소 초기화 완료")

d:\py_work\rag_test13\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pinecone 벡터 저장소 초기화 완료


In [3]:
test_query = "이 게임의 궁성 스킬은 어떤게 있어?"

docs = retriever.invoke(test_query)
print(f"검색 결과: {len(docs)}개의 문서")
print("-"*30)
for i, doc in enumerate(docs, 1):
    print(f"[문서 {i+1}] {doc.page_content[:100]}...")

d:\py_work\rag_test13\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


검색 결과: 3개의 문서
------------------------------
[문서 2] 문서 제목: 궁성 스킬
문서 요약: 궁성 클래스 스킬 정보입니다

내용:
마스터리 스킬
아이콘
명칭
설명
비고
저격 - 연사 - 나선 화살
적들에게 피해를 주고, 정신력을 회복합니...
[문서 3] 문서 제목: 마도성 스킬
문서 요약: 마도성 클래스 스킬 정보입니다

내용:
마스터리 스킬
아이콘
명칭
설명
비고
불꽃 화살 - 작렬 - 열화
대상에게 피해를 주고, 정신력을 회복...
[문서 4] 마스터리 스킬
아이콘
명칭
설명
비고
불꽃 화살 - 작렬 - 열화
대상에게 피해를 주고, 정신력을 회복합니다.
이동 가능
얼음 사슬 - 냉기 파동
대상을 중심으로 적들에게 물속성 ...


RAG 체인 생성 완료!


In [5]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_cohere import CohereRerank
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever


embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)


retriever = vector_store.as_retriever(search_kwargs={"k": 20})
# test_query = "이 게임의 궁성 스킬은 어떤게 있어?"

# docs = retriever.invoke(test_query)



compressor = CohereRerank(
    model="rerank-multilingual-v3.0", # 한국어 지원이 훌륭한 모델
    top_n=3 
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

query = "검성 스킬 콤보 알려줘"

final_docs = compression_retriever.invoke(query)

for doc in final_docs:
    print(f"점수: {doc.metadata['relevance_score']} | 내용: {doc.page_content[:50]}...")

점수: 0.2024432 | 내용: 검성
검성
높은 공격력과 방어력을 동시에 지닌 근접 전투의 핵심 클래스입니다.
검성은
대검...
점수: 0.09894659 | 내용: 문서 제목: 검성 스킬
문서 요약: 검성 클래스 스킬 정보입니다

내용:
마스터리 스킬
아...
점수: 0.031858854 | 내용: 문서 제목: 검성
문서 요약: 대검을 다루는 전장의 지휘관, 검성 클래스 소개입니다.

내...


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
당신은 게임 초보자를 도와주는 친절한 '게임 가이드 AI'입니다.
아래의 [게임 가이드북 내용]을 참고하여 질문에 대해 정확하고 이해하기 쉽게 답변해주세요.
가이드북에 없는 내용은 지어내지 말고 솔직하게 모른다고 답변하세요.

[게임 가이드북 내용]
{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print("RAG 체인 생성 완료!")

RAG 체인 생성 완료!


In [7]:
query = "검성 스킬에 대한 가이드를 알려줘"

print(f"질문: {query}")
print("답변 생성중...")
print("-"*50)

response = rag_chain.invoke(query)

print(response)

질문: 검성 스킬에 대한 가이드를 알려줘
답변 생성중...
--------------------------------------------------
검성 클래스의 스킬에 대한 가이드는 다음과 같습니다:

1. **마스터리 스킬**
   - **예리한 일격 / 파열의 일격 / 분노의 일격**: 적에게 피해를 주고 적대치를 증가시키며, 정신력을 회복합니다. 이 스킬은 논타겟이며 이동이 가능합니다.
   
2. **절단의 맹타 / 격파의 맹타**: 적에게 피해를 주고 적대치를 증가시키며, 그로기 게이지에 피해를 줍니다. 이 스킬은 논타겟입니다.

3. **도약 찍기**: 대상에게 도약하여 피해를 주며, 그로기 게이지에 피해를 줍니다.

4. **유린의 검**: 대상에게 피해를 주고 넘어짐 상태로 만듭니다. 행동불가 면역 대상일 경우 모션이 변경됩니다. 그로기 게이지에 피해를 줍니다.

5. **내려찍기**: 넘어짐 상태의 대상에게 피해를 줍니다.

6. **검기 난무**: 그로기 상태인 대상에게 피해를 줍니다.

7. **발목 베기**: 막기 성공 시 대상을 중심으로 적들에게 피해를 주고, 일정 확률로 속박 상태로 만듭니다. NPC일 경우 100% 확률로 속박이 적중됩니다. 그로기 게이지에 피해를 줍니다.

8. **분쇄 파동 / 광폭 파동**: 자신을 중심으로 적들에게 피해를 주며, 그로기 게이지에 피해를 줍니다. 이 스킬은 논타겟입니다.

9. **돌진 일격**: 긴급 회피 사용 후 대상에게 이동하여 피해를 주고, 일정 확률로 넘어짐 상태로 만듭니다. NPC일 경우 100%의 확률로 넘어짐이 적중됩니다. 그로기 게이지에 피해를 줍니다.

10. **공중 결박**: 넘어짐 상태의 대상에게 피해를 주고, 일정 확률로 공중속박 상태로 만듭니다. NPC일 경우 100%의 확률로 공중속박이 적중됩니다.

11. **파멸의 맹타**: 대상에게 돌진하여 피해를 주고 자신은 전투 준비 상태가 됩니다. 최대 차지 시 범위 피해를 줍니다. 전투 준비 상태에서는 피해 증폭 및 치

In [9]:
query = " 던전 초월 콘텐츠를 알려줘"

print(f"질문: {query}")
print("답변 생성중...")
print("-"*50)

response = rag_chain.invoke(query)

print(response)

질문:  던전 초월 콘텐츠를 알려줘
답변 생성중...
--------------------------------------------------
초월 던전은 일반 원정보다 높은 난이도로 제공되는 던전 콘텐츠입니다. 이 콘텐츠는 플레이어가 랭킹을 경쟁하며 즐길 수 있는 요소가 있으며, 보상으로 ‘돌파석 조각’과 ‘아르카나’를 획득할 수 있습니다. 초월 던전에 입장하기 위해서는 도전 횟수가 필요하며, 초월 보스를 클리어한 후 보상을 받을 때 도전 횟수가 소모됩니다. 만약 던전에서 클리어 후 보상을 받지 않으면 퇴장 시 도전 횟수는 소모되지 않습니다. 초월 도전 횟수는 매일 총 2회가 충전됩니다. 초월 단계는 1단계부터 10단계까지 있으며, 단계가 높아질수록 난이도와 보상이 상승합니다.
